In [28]:
import pandas as pd
import dodfminer as dod
from dodfminer.extract.polished.acts.aposentadoria \
    import RetAposentadoria, Retirements
import glob
import re
import gc
from itertools import zip_longest

### Carregando CSVs extraídos pelo DODFMiner

In [3]:
!ls

data	      extract-dodfminer.ipynb	      run-comparison.ipynb
date_num.csv  match-aposentadoria-ouro.ipynb


## Dando load nos .csv EXTRAÍDOS pelo DODFMiner

In [42]:
dfs_dodfminer = {}
for i in glob.glob('data/extraido/*.csv'):
    if 'sem_efeito' in i:
        continue
    fname = i.split('/')[-1]
    print("fname:", fname)
    k = re.search(r'(\d+)[_](\d+[-]\d+[-]+\d+)', fname).groups()
    k = ( k[1], str(int(k[0])))
    df = pd.read_csv(i, index_col='Unnamed: 0').rename(str.lower, axis=1)
    dfs_dodfminer[k] = df
    del df
gc.collect()

dfs_keys = list(dfs_dodfminer.keys())
dfs_keys[0]

fname: aposentadoria-093_20-05-2019.csv
fname: aposentadoria-248_01-01-2019.csv
fname: aposentadoria-074_22-04-2019.csv
fname: aposentadoria-043_01-03-2019.csv
fname: aposentadoria-127_09-07-2019.csv
fname: aposentadoria-047_12-03-2019.csv
fname: aposentadoria-181_21-09-2018.csv
fname: aposentadoria-108_10-06-2019.csv
fname: aposentadoria-209_01-11-2018.csv
fname: aposentadoria-186_30-09-2019.csv
fname: aposentadoria-187_01-10-2018.csv
fname: aposentadoria-156_19-08-2019.csv
fname: aposentadoria-114_18-06-2018.csv
fname: aposentadoria-147_03-08-2018.csv
fname: aposentadoria-175_13-09-2019.csv
fname: aposentadoria-068_10-04-2018.csv
fname: aposentadoria-117_25-06-2019.csv
fname: aposentadoria-124_03-07-2018.csv
fname: aposentadoria-108_08-06-2018.csv
fname: aposentadoria-010_15-01-2019.csv
fname: aposentadoria-197_16-10-2018.csv
fname: aposentadoria-180_20-09-2019.csv
fname: aposentadoria-066_06-04-2018.csv
fname: aposentadoria-085_04-05-2018.csv
fname: aposentadoria-105_05-06-2018.csv


('20-05-2019', '93')

## Dando load no .csv contendo 

In [43]:
df_ptr = pd.read_csv('data/date_num.csv')
df_ptr.drop('qtd', axis=1, inplace=True)
df_ptr['num'] = df_ptr.num.astype(str)

news = []
for i in df_ptr.loc[:, ('data', 'num')].iterrows():
    news.append(tuple(i[1].values))

In [44]:
for i in news:
    assert i in dfs_dodfminer

### Portanto todos os DODFs da base ouro (>= 50 ocorrencias) tiveram foram também extraídos via DODFMiner

## Dando load na BASE OURO

+ A partir dela extrair-se-á os atos referentes aos PDFs extraidos pelo DODFMiner

In [45]:
douro = pd.read_csv('data/aposentadoria-ouro/aposentaria-ouro-®.csv',
                    sep='®', engine='python')
douro.rename(str.lower, axis=1, inplace=True)
douro.rename(lambda x: x.replace('_dodf', ''), axis=1, inplace=True)
douro.rename(lambda x: x.replace('matricula', 'mat'), axis=1, inplace=True)
douro['data'] = douro.data.map(lambda x: '-'.join(x.split('-')[::-1]))
douro['nome_sigrh'] = douro.nome_sigrh.str.replace('\n', '' )
douro.ato.value_counts()

CONCEDER APOSENTADORIA    4750
APOSENTAR                  766
Name: ato, dtype: int64

In [46]:
douro.num

0       248
1       248
2       248
3       248
4       248
       ... 
5511    103
5512    103
5513    103
5514    103
5515    103
Name: num, Length: 5516, dtype: int64

### Testar se todas as chaves de dfs_dodfminer funcionam, pois todas serão usadas

In [47]:
for k in dfs_dodfminer:
    df = douro[(douro.data == k[0]) & (douro.num == int(k[1]))]
    assert df.shape[0] > 0

## Construção de dataframes padrão-ouro por numero e data

In [48]:
dfs_ouro = {}
for k in dfs_dodfminer:
        dfs_ouro[k] = douro[(douro.data == k[0]) & (douro.num == int(k[1]))]

In [54]:
for k in dfs_dodfminer:
    df_top = dfs_ouro[k]
    df_meh = dfs_dodfminer[k]
    true_names = list(sorted(df_top.nome_sigrh.dropna()))
    meh_names = list(sorted(df_meh.nome.dropna()))
    print('\t', *zip_longest(meh_names, true_names), sep='\n\t', end='\n\n\n')

	
	('ALESSANDRO FERREIRA FLORIANO', 'ALESSANDRO FERREIRA FLORIANO')
	('AMANDA DE OLIVEIRA ROCHA', 'AMANDA DE OLIVEIRA ROCHA')
	('ANA PAULA SANTOS ULHOA', 'ANA PAULA SANTOS ULHOA')
	('CLAUDETE CLAUDINO PEREIRA LOPES', 'CLAUDETE CLAUDINO P LOPES')
	('CLEIDE DE FATIMA MOREIRA DO LIVRAMENTO', 'CLEIDE DE FATIMA MOREIRA DO LIVRAMENTO')
	('CREUSA CASSARO DE SOUSA', 'CREUSA CASSARO DE SOUSA')
	('CRISTIANE DOS REIS GUEDES ANACLETO', 'CRISTIANE DOS REIS GUEDES ANACLETO')
	('CRISTINA BRAGA DE MENEZES', 'CRISTINA BRAGA DE MENEZES')
	('DIANA DUARTE VIEIRA', 'DIANA DUARTE VIEIRA')
	('ELAINE GUIMARAES RAMALHO', 'ELAINE GUIMARAES RAMALHO')
	('ELIANA PINTO DE OLIVEIRA', 'ELIANA PINTO DE OLIVEIRA')
	('ELIANE CRISTINA DE CARVALHO', 'ELIANE CRISTINA DE CARVALHO')
	('ELIFAS FARIAS DE LIMA', 'ELIFAS FARIAS DE LIMA')
	("ELISA TOMINAGA SANT'ANNA DE MORAES", "ELISA TOMINAGA SANT'ANNA DE MORAES")
	('ELKE MARIA PESSOA', 'ELKE MARIA PESSOA')
	('GERARDO COSMO DE OLIVEIRA', 'FRANCISCO DAS CHAGAS FILHO')
	('IVONE AL

In [22]:
# for k, df in dfs_dodfminer.items():
#     gold = douro[(douro.data == k[0]) & (douro.num == int(k[1]))]
#     print(df.shape, gold.shape)

## Extrair concessões de aposentadoria

In [184]:
retire = {}
for i in glob.glob('data/*.txt'):
#     print('file:', i)
    try:
        num = str(int(re.search('\d+', i).group()))
        data = re.search(r'\d{2}-\d{2}-\d{4}', i).group()
    except:
        continue
    _ = Retirements(i, 'regex')
    _.data_frame.rename(str.lower, axis=1, inplace=True)
    _.data_frame['text'] = _.acts_str
    retire[(data, num)] = _.data_frame
#     apos2 = RetAposentadoria(i, 'regex')
#     apos2.data_frame.rename(str.lower, axis=1, inplace=True)
#     break

In [186]:
retire[('20-05-2019', '93')].head(1)

,tipo do ato,sei,nome,matrícula,tipo de aposentadoria,cargo,classe,padrao,quadro,fundamento legal,orgao,vigencia,matricula siape,text
0,Aposentadoria,00040-00003703/2019-69,nan,43.465-5,nan,Tecnico de\nGestao Educacional/Apoio Administ...,nan,3,Quadro de Pessoal do\nDistrito Federal,nan,\nDistrito Federal,nan,nan,APOSENTAR FRANCISCO DAS CHAGAS FILHO matricula...


In [176]:
true_names = sorted(list(douro[(douro.num == 66) & (douro.data == '06-04-2018')].nome_sigrh))
names = sorted(apos1.data_frame.nome.map(lambda x: x.replace('\n', ' ')))
print(len(true_names), ' true names')
print(len(names), ' names')

print("TRUE\t\t\t\t\t\tCAND")
for idx, tp in enumerate(zip(true_names, names)):
    print(idx, end=' ')
    print(*tp, sep=' |********| ')

51  true names
51  names
TRUE						CAND
0 ALCIZIO HENRIQUE LEE LIMA DOS SANTOS |********| ALCIZIO HENRIQUE LEE LIMA DOS SANTOS
1 ALENCAR FERNANDES ALVES |********| ALENCAR FERNANDES ALVES
2 ANA CRISTINA FERREIRA DO NASCIMENTO |********| ANA CRISTINA FERREIRA DO NASCIMENTO
3 ANASTACIO NERES PORTELA |********| ANASTACIO NERES PORTELA
4 ANDREIA CRISTINA CRUZ |********| ANDREIA CRISTINA CRUZ
5 ANTONIETA BARROS DE A DIAS |********| ANTONIETA BARROS DE ARAUJO DIAS
6 BENEDITA MARTINS DA SILVA |********| BENEDITA MARTINS DA SILVA
7 CELIA DA SILVA CORREA LOPES |********| CELIA DA SILVA CORREA LOPES
8 CLAUDIA G P L DAS CHAGAS |********| CLAUDIA GONCALVES PORTELA LISBOA DAS CHAGAS
9 CLEONICE MARIA PINTO MAGALHAES |********| CLEONICE MARIA PINTO MAGALHAES
10 CONCEICAO ROSEMIR DE PAULA PINTO |********| EDINA CHAVES ARAUJO
11 EDINA CHAVES ARAUJO |********| FERNANDA CRISTINA PARENTE ROSAL
12 FERNANDA CRISTINA PARENTE ROSAL |********| HERMINIA MARQUES DE SOUSA
13 HERMINIA MARQUES DE SOUSA |********| J

In [177]:
names[8]

'CLAUDIA GONCALVES PORTELA LISBOA DAS CHAGAS'

In [152]:
true_names

['ADINA VIEIRA MOUHAMAD ABOU',
 'ALCIZIO HENRIQUE LEE LIMA DOS SANTOS',
 'ALENCAR FERNANDES ALVES',
 'ANA CRISTINA FERREIRA DO NASCIMENTO',
 'ANASTACIO NERES PORTELA',
 'ANDREIA CRISTINA CRUZ',
 'ANTONIETA BARROS DE A DIAS',
 'AURIO SOLON DE OLIVEIRA SOUZA',
 'BENEDITA MARTINS DA SILVA',
 'CAROLINE MARIA SOARES RIBEIRO ALCANTARA',
 'CELIA DA SILVA CORREA LOPES',
 'CLAUDIA BARBOSA CARRILHO PEREIRA',
 'CLAUDIA G P L DAS CHAGAS',
 'CLAUDIO SEVERIANO RAMOS',
 'CLEONICE MARIA PINTO MAGALHAES',
 'CONCEICAO ROSEMIR DE PAULA PINTO',
 'EDILMAENI DE JESUS CONCEICAO SILVA',
 'EDINA CHAVES ARAUJO',
 'FERNANDA CRISTINA PARENTE ROSAL',
 'FRANCISCO PORFIRIO DA SILVA',
 'HERMINIA MARQUES DE SOUSA',
 'IVONE FERNANDES GUIMARAES',
 'JADIR DA SILVA BOTELHO',
 'JANETE APARECIDA ROQUE DE ALMEIDA',
 'JAQUELINE SOUZA ALMEIDA',
 'JOAO BATISTA GONCALVES',
 'JOSE MARCILIO LEITE DOS PRAZERES',
 'JOSE RAUL DE OLIVEIRA VIEIRA',
 'JOSE ROMUALDO FERREIRA DA CRUS',
 'JOSE VALDINO DE ALMEIDA',
 'JULIA SANTOS DO NASCIME